In [1]:
import time
import os
from collections import defaultdict
import sqlite3
import shutil
import json
import string
import math
import statistics

UPDATED_ENGLISH_FILE = 'English_Updated.txt'
UPDATED_DUTCH_FILE = 'Dutch_Updated.txt'

TRAINING_ENG_FILE = 'eng_file.txt'
TRAINING_DUTCH_FILE = 'dutch_file.txt'

tr_etof = {}
tr_ftoe = {}

con = sqlite3.connect('TransProb_100.db')
crsr = con.cursor()

eng_data = []
dutch_data = []

eng_cleaned = []
dut_cleaned = []

total_no_of_sentences = -1

dutchWords_line_no = defaultdict(set)
engWords_line_no = defaultdict(set)


num_dict_eng = {}
num_dict_dutch = {}

total = {}


In [2]:
def EngToDutchStopWords():
    eng_sw=['of', 'the', 'I', 'on', 'and', 'would', 'to', 'you', 'a', 'in', 'that', 'as', 'have', 'for', 'be', 'from', 'it', 'at', 'can', 'an', 'has', 'The', 'It', 'is', 'not', 'with', 'We', 'by', 'This', 'we', 'are', 'more', 'our', 'or', 'also', 'these', 'but', 'must']
    dut_sw=['van', 'de', 'ik', 'Aan', 'en', 'Zou', 'naar', 'u', 'een', 'in', 'dat', 'als', 'hebben', 'voor', 'worden', 'van', 'het', 'Bij', 'kan', 'een', 'heeft', 'De', 'Het', 'is', 'niet', 'met', 'Wij', 'door', 'Deze', 'wij', 'zijn', 'meer', 'onze', 'of', 'ook', 'deze', 'maar', 'moet']
    eng_sw = [i.lower() for i in eng_sw]
    dut_sw = [i.lower() for i in dut_sw]
    
    return (dict(zip(eng_sw, dut_sw)))


In [3]:
def DutchToEngStopWords():
    eng_sw=['of', 'the', 'I', 'on', 'and', 'would', 'to', 'you', 'a', 'in', 'that', 'as', 'have', 'for', 'be', 'from', 'it', 'at', 'can', 'an', 'has', 'The', 'It', 'is', 'not', 'with', 'We', 'by', 'This', 'we', 'are', 'more', 'our', 'or', 'also', 'these', 'but', 'must']
    dut_sw=['van', 'de', 'ik', 'Aan', 'en', 'Zou', 'naar', 'u', 'een', 'in', 'dat', 'als', 'hebben', 'voor', 'worden', 'van', 'het', 'Bij', 'kan', 'een', 'heeft', 'De', 'Het', 'is', 'niet', 'met', 'Wij', 'door', 'Deze', 'wij', 'zijn', 'meer', 'onze', 'of', 'ook', 'deze', 'maar', 'moet']
    eng_sw = [i.lower() for i in eng_sw]
    dut_sw = [i.lower() for i in dut_sw]
    
    return (dict(zip( dut_sw,eng_sw)))

In [4]:
def readfiles(Eng_File, Dutch_File,no_of_sentences):
    """
    Function to read required number of lines(no_of_sentences) from given text files  
    
    """
    e_file = open(Eng_File,'r',encoding = 'utf-8')
    d_file = open(Dutch_File,'r',encoding = 'utf-8')
    
    global dutch_data
    global eng_data
    dutch_data = d_file.readlines()[:no_of_sentences]
    eng_data = e_file.readlines()[:no_of_sentences]
    print(eng_data)

In [5]:
def remove_punc(l,lang):
    """
    Function to remove punctuation symbols from acquired sentences

    """
    
    for i in range(len(l)):
        l[i] = l[i][:-1].lower()
        l[i] = remove_stopwords(l[i],lang)
        l[i] = remove_stuff(l[i])
    return l

#CHECKED
def remove_stopwords(l,lang):
    eng_sw=['of', 'the', 'i', 'on', 'and', 'would', 'to', 'you', 'a', 'in', 'that', 'as', 'have', 'for', 'be', 'from', 'it', 'at', 'can', 'an', 'has', 'the', 'it', 'is', 'not', 'with', 'we', 'by', 'this', 'we', 'are', 'more', 'our', 'or', 'also', 'these', 'but', 'must']
    dut_sw=['van', 'de', 'ik', 'aan', 'en', 'zou', 'naar', 'u', 'een', 'in', 'dat', 'als', 'hebben', 'voor', 'worden', 'van', 'het', 'bij', 'kan', 'een', 'heeft', 'de', 'het', 'is', 'niet', 'met', 'wij', 'door', 'deze', 'wij', 'zijn', 'meer', 'onze', 'of', 'ook', 'deze', 'maar', 'moet']
    eng_sw = [i.lower() for i in eng_sw]
    dut_sw = [i.lower() for i in dut_sw]
    
    words = l.split()
    s = []
    
    if lang =='dutch':
        for i in words:
            if i not in dut_sw:
                s.append(i)
    elif lang =='eng':
        for i in words:
            if i not in eng_sw:
                s.append(i)
    return ' '.join(s)

#CHECKED    
def remove_stuff(l):
    #a = [ ':','.' , '\\' , '/' , ',' , ';' , '(' , ')' , '"', "\'",'1','2','3','4','5','6','7','8','9','0','?']
    for i in string.punctuation + string.digits:
        l = l.replace(i, "")
    return l


In [6]:
def assign_line_no(doc):
    dict_lo = defaultdict(set)
    for i in range(len(doc)):
        t = doc[i].split()
        for m in t:
            dict_lo[m].add(i)
    return dict_lo

In [7]:
def finding_probabilities():
    """
    Function to update count for each english-dutch pair
    
    """

    global total
    global eng_cleaned
    global dut_cleaned
    global total_no_of_sentences


    for i in range(total_no_of_sentences):
        
        
        en = eng_cleaned[i]
        en_words = en.split()
        

        du = dut_cleaned[i]
        du_words = du.split()
        

        retrieved_count = {}
        retrieved_term_probability = {}

        for e in en_words:

            for d in du_words:

                comm = "select * from TransProb where ENG_WORD= '{0}' AND DUT_WORD= '{1}'".format(e,d)
                com2 = "select * from Count where ENG_WORD= '{0}' AND DUT_WORD= '{1}'".format(e,d)
                #print(comm)
                #print(com2)
                
                
                crsr.execute(comm)
                ans = crsr.fetchall()
                
                crsr.execute(com2)
                an2 = crsr.fetchall()
                try:
                    line_no2= ans[0][0]+'_'+ans[0][1]+' '+str(ans[0][2])
                    line_no = an2[0][0]+'_'+an2[0][1]+' '+str(an2[0][2])
                except:
                    print(line_no2)
                pr = line_no.split()
                pr2 = line_no2.split()
                
                #print(pr)
                #print(pr2)
                retrieved_count[pr[0]] = float(pr[1])
                retrieved_term_probability[pr2[0]] = float(pr2[1])
              

        print("Count and Translation probabilities retrieved")
        
        #print(retrieved_term_probability)
        print()

        s_total = {}

        for e in en_words:

            s_total[e] = 0

            for d in du_words:

                s = e+'_'+d

                s_total[e] += retrieved_term_probability[s]

       #print(s_total)
        print("S_total for each english word done")
                
        for e in en_words:

            for d in du_words:

                s = e+'_'+d

                retrieved_count[s] += (retrieved_term_probability[s]/s_total[e])

                total[d] += (retrieved_term_probability[s]/s_total[e])

        print("Counts modified and Total calculated")
        #print(retrieved_count)
        print()
        print("RETRIEVED TP")
        #print(retrieved_term_probability)
        print()
        
        for k,v in retrieved_count.items():
            
            t = k.split('_')
            e = t[0]
            d = t[1]
            #print(k)
            #print(e+" "+d)
            
            command= "UPDATE Count SET PROBABILITY={0} WHERE ENG_WORD = '{1}' AND DUT_WORD ='{2}' ".format(v,e,d)
            #print(command)
            crsr.execute(command)
                
        con.commit()
        
        print("Writeback completed into count file")

In [8]:
def running_function(no_of_iterations=10):
    """
    Function to reinitialize total for foreign words and count for eng-dutch pairs. It carries out (no_of_iterations) iterations
    It rewrites the translational probabilities back into the CondProb file.
    """
    
    global total
    global dutchWords_line_no
    global engWords_line_no
    global total_no_of_sentences
    global dut_cleaned
    global eng_cleaned



    for c_i in range(no_of_iterations):
        
        print(c_i+1)
        
        for i in engWords_line_no.keys():
            for j in dutchWords_line_no.keys():
                command= "UPDATE Count SET PROBABILITY= {0} WHERE ENG_WORD = '{1}' AND DUT_WORD ='{2}' ".format(0,i,j)
                #print(command)
                crsr.execute(command)
            con.commit()
            
                
    
        total = {}
    
        for k in dutchWords_line_no.keys():
            total[k] = 0
    
        finding_probabilities()
    
    
        print("Finding probabilities done, ",c_i)
        retrieved_count = {}
        retrieved_term_probability = {}
    
       # print(total)
        
        
        for e in engWords_line_no.keys():
            
            for d in dutchWords_line_no.keys():
                
                com2 = "select * from Count where ENG_WORD= '{0}' AND DUT_WORD= '{1}'".format(e,d)
                #print(com2)
                crsr.execute(com2)
                ans = crsr.fetchall()
                
                try:
                    line_no = ans[0][0]+'_'+ans[0][1]+' '+str(ans[0][2])
                except:
                    print(line_no)
                
                pr = line_no.split()
                
                retrieved_count[pr[0]] = float(pr[1])
            con.commit()
        
    

        print("Counts, retrieved")
        
        print()
        #print(retrieved_count)
        print()
    
        for d in dutchWords_line_no.keys():
            for e in engWords_line_no.keys():
            
                s = e+ '_' +d
                retrieved_term_probability[s] = retrieved_count[s]/total[d]  
                
                
                command= "UPDATE TransProb SET PROBABILITY= {0} WHERE ENG_WORD = '{1}' AND DUT_WORD ='{2}' ".format(float(retrieved_term_probability[s]),e,d)
                #print(command)
                crsr.execute(command)
                
            con.commit()
        
        print("Translational Probabilites updated")
        

In [9]:
def retrieve_max(english_l):
    """
    Function to retieve the translaton with maximum probabilities for each word to be translated, i.e. to obtain the most probable translation for each word.     
    """
    global tr_etof
    global tr_ftoe
    for e in english_l.keys():
        
        #print(e,counter)
        
        comm = "SELECT MAX(PROBABILITY),DUT_WORD From TransProb where ENG_WORD = '{0}'".format(e)
        
        crsr.execute(comm)
        
        ans = crsr.fetchall()
         
        tr_etof[e] = ans[0][1]
        tr_ftoe[ans[0][1]] = e

In [45]:
def pearson_coefficient(dut_cleaned,result_dut):
    """
    Function to find Pearson Coefficient
    """
    global dutchWords_line_no  
    total_sim=0
    
    resultDutWords_line_no = assign_line_no(result_dut)
    dutchWord_line_no = assign_line_no(dut_cleaned)

    tf_of_cleaned = maintain_count(dutchWord_line_no,dut_cleaned)
    
    tf_of_result = maintain_count(resultDutWords_line_no,result_dut)
    
    avg_cleaned = statistics.mean(tf_of_cleaned.values())
    avg_result = statistics.mean(tf_of_result.values())
    
    print(tf_of_cleaned)
    print(tf_of_result)
    
    
    for i in tf_of_cleaned.keys():
        
        total_sim += ((tf_of_cleaned[i]-avg_cleaned) * (tf_of_result[i] - avg_result))
        
    stddev_cleaned = statistics.stdev(tf_of_cleaned.values())
    stddev_result = statistics.stdev(tf_of_result.values())
    
    print(avg_cleaned,avg_result)
    
    print(stddev_cleaned,stddev_result)
    
    print(total_sim)
    
    return total_sim/(stddev_cleaned*stddev_result*len(tf_of_cleaned))

In [40]:
def maintain_count(doc_dict_line_no,dut_cleaned):
    
    og_dutch_tf = {}
    for dword in doc_dict_line_no:                #for each word in the document
        dlist=[]
        dlist=list(doc_dict_line_no[dword])
        count = 0
        for i in dlist:
            count+= dut_cleaned[i].count(dword)
        
        og_dutch_tf[dword] = (count)

    return og_dutch_tf

In [41]:
def cosine_similarity(dut_cleaned,result_dut):
    """
    Function to find cosine similarity between 2 documents
    
    Required: Clean test data before calculating cosine_similarity
    """
    global dutchWords_line_no
    
    resultDutWords_line_no = assign_line_no(result_dut)
    dutchWord_line_no = assign_line_no(dut_cleaned)

    tf_of_cleaned = maintain_normalized_tf(dutchWord_line_no,dut_cleaned)
    
    tf_of_result = maintain_normalized_tf(resultDutWords_line_no,result_dut)
    
    total_sim = 0
    
    print(tf_of_cleaned)
    print(tf_of_result)
    
    
    for i in tf_of_cleaned.keys():
        total_sim += tf_of_cleaned[i] * tf_of_result[i]
        
    return total_sim

In [28]:
def maintain_normalized_tf(doc_dict_line_no,dut_cleaned):
    
    """
    Function to find normalized term-frequency of a word in a document.
    
    """
    og_dutch_tf = {}
    
    sum_of_tf = 0
    
    for dword in doc_dict_line_no:                #for each word in the document
        dlist=[]
        dlist=list(doc_dict_line_no[dword])
        count = 0
        for i in dlist:
            count+= dut_cleaned[i].count(dword)
        
        og_dutch_tf[dword] = math.log(count)+1
        sum_of_tf += pow(og_dutch_tf[dword],2)
    
    normalized_denom = pow(sum_of_tf,0.5)
    
    for i in og_dutch_tf.keys():
        og_dutch_tf[i] = og_dutch_tf[i]/normalized_denom

    return og_dutch_tf

In [14]:
def produce_sentence(eng_sentence, translated_dict,doc_lang):
    """
    Function to translate a sentence.
    """
    
    s = ""

    
    for i in eng_sentence:
        if i in string.punctuation + string.digits:
            s+=i
            continue
            
        s += str(tr[i])
    
    s+='\n'
    
    return s

In [15]:
def translate_doc(eng_doc, translated_dict, rewrite_file, doc_lang = 'eng'):
    """
    Function to translate a document.
    """

    
    result_doc = []
    
    for i in eng_doc:
        s = produce_sentence(i, translated_dict,doc_lang)
        result_doc.append(s)
    
    with open(rewrite_file,'w') as f:
        f.writelines(result_doc)
    
    return result_doc


In [16]:
def main():
    
    
    Eng_File = TRAINING_ENG_FILE
    Dutch_File = TRAINING_DUTCH_FILE
    

    readfiles(Eng_File,Dutch_File,50)
    
    print("Files read")
    
    # Cleaning Data

    global eng_cleaned
    global dut_cleaned
    
    eng_cleaned = remove_punc(eng_data,'eng')
    dut_cleaned = remove_punc(dutch_data,'dutch')

    global total_no_of_sentences
    
    total_no_of_sentences = len(dut_cleaned)

    assert(total_no_of_sentences!=-1)
    
    print("Files cleaned")

    global dutchWords_line_no
    global engWords_line_no
    
    dutchWords_line_no = assign_line_no(dut_cleaned)
    engWords_line_no = assign_line_no(eng_cleaned)
    
    no_of_words_eng = len(engWords_line_no)
    no_of_words_dutch = len(dutchWords_line_no)
    
    print("Initialization Done")
    
    running_function(10)
    
    print("Model Trained")
    
    global tr_etof
    global tr_ftoe

    retrieve_max(engWords_line_no)
    
    tr_etof.update(EngToDutchStopWords())
    tr_ftoe.update(DutchToEngStopWords())
    
    print(tr_etof)
    print("\n\n\n\n")
    print(tr_ftoe)
    
    with open('etof.json','w') as fp:
        json.dump(tr_etof,fp)
    
    with open('ftoe.json','w') as fp:
        json.dump(tr_ftoe,fp)
    
    #new_doc = open(TEST_FILE,'r+')
    #lines = new_doc.readlines()
    
    #b = translate_doc(lines,tr_etof,TEST_FILE)
    
    #print(b)
    

In [17]:
if __name__ == "__main__":
    main()

['Relating to Wednesday:\n', 'Thank you very much.\n', 'My vote was "in favour" .\n', 'That did not happen.\n', 'The debate is closed.\n', 'The debate is closed.\n', 'The debate is closed.\n', 'What is the result?\n', 'The debate is closed.\n', 'Are there any comments?\n', 'Thank you very much, Mr Cox.\n', 'Thank you very much.\n', 'But Europe is changing.\n', 'This is not in question.\n', 'I do not believe so.\n', 'Just like Europol.\n', 'Oil slick off the French coast\n', 'Why not?\n', 'Next, we have flags of convenience.\n', 'This is no longer tolerable.\n', 'This is no longer tolerable.\n', 'Many thanks for this.\n', 'But this is not in place.\n', 'So what are we to do now?\n', 'I cannot accept this.\n', 'They want answers.\n', 'That is the reality.\n', 'The debate is closed.\n', 'Storms in Europe\n', 'Let me explain.\n', 'That is important.\n', 'That concludes the debate.\n', 'Food safety\n', 'That cannot continue.\n', 'It must be visible.\n', 'It must be known.\n', 'That conclude

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calcul

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calcul

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calcul

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calcul

...

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calcul

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calcul

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calcul

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calcul

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calculated

RETRIEVED TP

Writeback completed into count file
Count and Translation probabilities retrieved

S_total for each english word done
Counts modified and Total calcul

In [46]:
#print(dutch_data2)
#print(dut_cleaned)
pearson_coefficient(dut_cleaned,dut_cleaned)

{'woensdag': 1, 'januar': 1, 'hartelijk': 1, 'dan': 1, 'was': 1, 'vóó': 1, 'dit': 4, 'gebeur': 1, 'debat': 8, 'geslote': 8, 'wat': 2, 'gevol': 1, 'geen': 2, 'bezware': 1, 'mijnheer': 1, 'cox': 1, 'dank': 3, 'zee': 1, 'europa': 1, 'verander': 1, 'daarover': 1, 'discussi': 1, 'denk': 1, 'nie': 6, 'net': 1, 'europo': 1, 'olievlek': 1, 'frankrij': 1, 'hoe': 1, 'komt': 2, 'da': 1, 'mee': 3, 'zeer': 1, 'veel': 1, 'daarvoo': 1, 'er': 1, 'moeten': 1, 'nu': 1, 'doe': 1, 'aanvaard': 1, 'zij': 5, 'willen': 1, 'antwoor': 1, 'fei': 1, 'noodweer': 1, 'europ': 1, 'verklaar': 1, 'mij': 1, 'nade': 1, 'belangrij': 1, 'voedselveilighei': 1, 'afgelopen': 1, 'zichtbaar': 1, 'bekend': 1, 'vragenuur': 1, 'commissi': 1, 'eerste': 1, 'dee': 1, 'deel': 1, 'ze': 1, 'zo': 1, 'niet': 1, 'waarom': 1, 'vaak': 1, 'voo': 1, 'gefeliciteer': 1, 'kwijting': 1, 'die': 1, 'we': 1, 'al': 1, 'binne': 1, 'laat': 1, 'ons': 1, 'eerlijk': 1, 'tenslotte': 1, 'nog': 1, 'di': 1, 'vraag': 1, 'alleen': 1, 'ho': 1, 'overdreven': 1, 'u

0.9878048780487779

In [24]:
eng_data = remove_punctuation(eng_data)
dutch_data2 = remove_punctuation(dutch_data)
Eng_File = TRAINING_ENG_FILE
Dutch_File = TRAINING_DUTCH_FILE
readfiles(Eng_File,Dutch_File,50)
dutch_data2

['Relating to Wednesday:\n', 'Thank you very much.\n', 'My vote was "in favour" .\n', 'That did not happen.\n', 'The debate is closed.\n', 'The debate is closed.\n', 'The debate is closed.\n', 'What is the result?\n', 'The debate is closed.\n', 'Are there any comments?\n', 'Thank you very much, Mr Cox.\n', 'Thank you very much.\n', 'But Europe is changing.\n', 'This is not in question.\n', 'I do not believe so.\n', 'Just like Europol.\n', 'Oil slick off the French coast\n', 'Why not?\n', 'Next, we have flags of convenience.\n', 'This is no longer tolerable.\n', 'This is no longer tolerable.\n', 'Many thanks for this.\n', 'But this is not in place.\n', 'So what are we to do now?\n', 'I cannot accept this.\n', 'They want answers.\n', 'That is the reality.\n', 'The debate is closed.\n', 'Storms in Europe\n', 'Let me explain.\n', 'That is important.\n', 'That concludes the debate.\n', 'Food safety\n', 'That cannot continue.\n', 'It must be visible.\n', 'It must be known.\n', 'That conclude

['woensdag  januari',
 'hartelijk dank',
 'ik was vóór',
 'dit is niet gebeurd',
 'het debat is gesloten',
 'het debat is gesloten',
 'het debat is gesloten',
 'wat is het gevolg',
 'het debat is gesloten',
 'geen bezwaren',
 'mijnheer cox ik dank u',
 'ik dank u zeer',
 'maar europa verandert',
 'daarover geen discussie',
 'ik denk van niet',
 'net als europol',
 'olievlek in frankrijk',
 'hoe komt dat',
 'dit kan niet meer',
 'dit kan niet meer',
 'dit kan niet meer',
 'zeer veel dank daarvoor',
 'maar dat is er niet',
 'wat moeten wij nu doen',
 'ik aanvaard dat niet',
 'zij willen een antwoord',
 'dat is een feit',
 'het debat is gesloten',
 'noodweer in europa',
 'ik verklaar mij nader',
 'dat is belangrijk',
 'het debat is gesloten',
 'voedselveiligheid',
 'dat moet afgelopen zijn',
 'het moet zichtbaar zijn',
 'het moet bekend zijn',
 'het debat is gesloten',
 'vragenuur commissie',
 'eerste deel',
 'ik deel ze niet',
 'zo niet waarom niet',
 'dat komt niet vaak voor',
 'gefelic

In [19]:
def remove_punctuation(l):
    """
    Function to remove punctuation symbols from acquired sentences

    """
    
    for i in range(len(l)):
        l[i] = l[i][:-1].lower()
        l[i] = remove_stuff(l[i])
    return l

In [ ]:
dutch_data2